# RAG 02: Decomposing the retrieval process

This example demonstrates query expansion and multi-step retrieval using pydantic-ai embeddings and Chroma.
Uses the LLM-chunked collection from example_RAG_02_load.ipynb.

## Initialize

In [ ]:
from pathlib import Path

from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.vectordb import get_vector_db, vdb_query

## Vector-db: Load existing collection

Assumes the 'books_llm_chunked' collection was populated by running example_RAG_02_load.ipynb first.

In [ ]:
vdb = get_vector_db('books_llm_chunked')

In [ ]:
# Check database has documents
count = vdb.count()
assert count > 0, "Vector database is empty, please run example_RAG_02_load.ipynb first to populate it."
print(f"Collection has {count} documents")

## RAG

In [ ]:
query = "Who is a man with two heads?"

### Query expansion

In [ ]:
prompt = f"""
Given the following user query, reformulate the query in three to five different ways to retrieve relevant documents from the vector database.

{query}
"""

In [ ]:
agent = get_agent(output_type=list[str]) # type: ignore
reformulated_queries, nodes = await run_agent(agent, prompt=prompt, verbose=True)


print(f"\nAnswer (len {len(reformulated_queries)}):")
for i, query in enumerate(reformulated_queries):
    print(f"{i+1:2d}: {query}")

### Query vector database

In [ ]:
documents_with_scores = []
for query in reformulated_queries:
    print(f"Query: {query}")
    # Query the vector database - returns (doc, meta, score) tuples
    documents_with_scores.extend(vdb_query(vdb, query=query))

print(f"\nFound {len(documents_with_scores)} documents with scores")

### Filter results

In [ ]:
# Deduplicate by creating a unique key from source and chunk
seen_ids = set()
documents_with_scores_filtered = []
for doc, meta, score in documents_with_scores:
    doc_id = f"{meta['source']}-{meta['chunk']}"
    if doc_id in seen_ids:
        continue
    documents_with_scores_filtered.append((doc, meta, score, doc_id))
    seen_ids.add(doc_id)
print(f"Filtered to {len(documents_with_scores_filtered)} unique documents")

### Metadata filtering

In [ ]:
book_name = 'hhgttg'
documents_with_scores_filtered_meta = []
for doc, meta, score, doc_id in documents_with_scores_filtered:
    if book_name in meta['source']:
        documents_with_scores_filtered_meta.append((doc, meta, score, doc_id))
print(f"Filtered to {len(documents_with_scores_filtered_meta)} documents from '{book_name}'")

### Re-ranking

In [ ]:
# Trivial "re-ranking" by score (index 2 is the score)
documents_with_scores_reranked = sorted(documents_with_scores_filtered_meta, key=lambda x: x[2], reverse=True)

max_results = 10
if len(documents_with_scores_reranked) > max_results:
    documents_with_scores_reranked = documents_with_scores_reranked[:max_results]

print(f"Re-ranked to top {len(documents_with_scores_reranked)} documents")

### Add results to prompt

In [ ]:
docs_str = ''
for doc, meta, score, doc_id in documents_with_scores_reranked:
    docs_str += f"Similarity Score: {score:.3f}\nDocument ID: {doc_id}\nDocument:\n{doc}\n\n"
    text = doc.replace('\n', ' ')
    print(f"Score: {score:.3f}, ID: {doc_id}, Document: {text[:80]}...")

### Prompt: Grounding on retrieved documents

In [ ]:
prompt = f"""
Given the following documents, answer the user's question.
Show used references (using document ids).

## Documents

{docs_str}

## User's question

{query}

"""

print(prompt[:1000])  # Print the first 1000 characters of the prompt

### Query the LLM with the vdb resuts

In [ ]:
agent = get_agent()
answer, nodes = await run_agent(agent, prompt=prompt, verbose=True)
print(f"\nAnswer: {answer}")